# Crumple Video - Blender XPBD Paper Animation

Генерация видеоанонса с эффектом раскрывающейся бумаги.

**Особенности:**
- Blender XPBD физика на CPU
- Обложка с датами и типографикой
- Аудиотрек The xx - Intro (старт с 1:10)

**Версия из репозитория events-bot-new**

In [ ]:
NOTEBOOK_VERSION = 'v1.0-crumple'
print(f'🚀 Engine {NOTEBOOK_VERSION}')

# ==========================================
# 1. УСТАНОВКА ЗАВИСИМОСТЕЙ
# ==========================================
import warnings
warnings.filterwarnings("ignore")

print("⏳ Installing dependencies...", flush=True)
!apt-get update -qq && apt-get install -y -qq blender ffmpeg > /dev/null 2>&1
!pip install opencv-python numpy pillow requests telethon > /dev/null

import os
import sys
import json
import glob
import math
import shutil
import subprocess
import random
import time
from pathlib import Path
from dataclasses import dataclass

import numpy as np
import cv2
from PIL import Image, ImageFont, ImageDraw

def log(msg):
    print(msg, flush=True)

log("✅ Dependencies ready")

In [ ]:
# ==========================================
# 2. КОНФИГУРАЦИЯ
# ==========================================
KAGGLE_INPUT_ROOT = "/kaggle/input"
WORKING_DIR = "/kaggle/working"

# Ищем папки сессии
session_dirs = sorted(glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "video-afisha-session-*")), reverse=True)

if session_dirs:
    SOURCE_FOLDER = session_dirs[0]
    log(f"✅ DETECTED BOT SESSION: {SOURCE_FOLDER}")
elif os.path.exists(os.path.join(KAGGLE_INPUT_ROOT, "afisha-dataset-2")):
    SOURCE_FOLDER = os.path.join(KAGGLE_INPUT_ROOT, "afisha-dataset-2")
    log(f"⚠️ Using legacy dataset: {SOURCE_FOLDER}")
else:
    all_dirs = [d for d in glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "*")) if os.path.isdir(d)]
    SOURCE_FOLDER = all_dirs[0] if all_dirs else "/kaggle/input/afisha-dataset-2"
    log(f"⚠️ Fallback: {SOURCE_FOLDER}")

# Параметры видео
W, H = 1080, 1920  # Вертикальный формат
FPS = 24

# Аудио
AUDIO_START_SEC = 70  # 1:10 от начала трека
AUDIO_FILENAME = "The_xx_-_Intro.mp3"

# Blender params
FRAME_START = 33
FRAME_END = 44
BLENDER_SAMPLES = 24  # Reduced for faster CPU rendering

# Дефолтный payload
DEFAULT_RAW_DATA = {
    "intro": {"count": 5, "text": "СОБЫТИЯ КОТОРЫЕ СТОИТ ПОСЕТИТЬ", "date": "25-27 декабря"},
    "scenes": [
        {"about": "Тестовое мероприятие", "date": "25 декабря", "location": "Калининград",
         "images": ["https://files.catbox.moe/vg30ot.jpg"]}
    ]
}

os.makedirs(WORKING_DIR, exist_ok=True)
log("✅ Config ready")

In [ ]:
# ==========================================
# 3. ГЕНЕРАТОР ОБЛОЖКИ
# ==========================================

def find_font(name: str) -> str:
    """Ищет шрифт в SOURCE_FOLDER и стандартных путях."""
    candidates = [
        os.path.join(SOURCE_FOLDER, name),
        os.path.join(SOURCE_FOLDER, "fonts", name),
        f"/usr/share/fonts/truetype/{name}",
    ]
    # Также ищем глобально
    candidates.extend(glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "**", name), recursive=True))
    for p in candidates:
        if os.path.exists(p):
            return p
    return None

def generate_cover_image(date_range: str, month: str, output_path: str) -> str:
    """
    Генерирует обложку в стиле макета:
    - Жёлтый фон #F5E10E
    - Даты крупно (Benzin-Bold)
    - Текст "СОБЫТИЯ КОТОРЫЕ СТОИТ ПОСЕТИТЬ" разными шрифтами
    - Месяц вертикально справа
    """
    # Размеры как в рендере Blender
    width, height = 864, 1104
    
    # Цвета
    bg_color = (245, 225, 14)  # #F5E10E жёлтый
    text_color = (16, 14, 14)  # Почти чёрный
    
    img = Image.new('RGB', (width, height), bg_color)
    draw = ImageDraw.Draw(img)
    
    # Загружаем шрифты
    font_benzin = find_font("Benzin-Bold.ttf")
    font_bebas = find_font("BebasNeue-Regular.ttf")
    font_oswald = find_font("Oswald-VariableFont_wght.ttf")
    font_druk = find_font("DrukCyr-Bold.ttf")
    
    def load_font(path, size):
        if path and os.path.exists(path):
            return ImageFont.truetype(path, size)
        return ImageFont.load_default()
    
    # Даты "25-27" - крупно сверху
    font_dates = load_font(font_benzin, 200)
    draw.text((50, 80), date_range, font=font_dates, fill=text_color)
    
    # "СОБЫТИЯ" - Bebas Neue, наклон
    font_events = load_font(font_bebas, 100)
    draw.text((350, 340), "СОБЫТИЯ", font=font_events, fill=text_color)
    
    # "КОТОРЫЕ" - Oswald, крупно
    font_which = load_font(font_oswald, 130)
    draw.text((50, 450), "КОТОРЫЕ", font=font_which, fill=text_color)
    
    # Месяц вертикально справа
    font_month = load_font(font_bebas, 90)
    month_upper = month.upper()
    # Создаём отдельное изображение для вертикального текста
    month_img = Image.new('RGBA', (200, 600), (0, 0, 0, 0))
    month_draw = ImageDraw.Draw(month_img)
    month_draw.text((0, 0), month_upper, font=font_month, fill=text_color)
    month_rotated = month_img.rotate(90, expand=True)
    # Вставляем повёрнутый месяц справа
    img.paste(month_rotated, (width - 150, 300), month_rotated)
    
    # "СТОИТ" - Bebas Neue
    font_worth = load_font(font_bebas, 100)
    draw.text((300, 600), "СТОИТ", font=font_worth, fill=text_color)
    
    # "ПОСЕТИТЬ" - Druk Cyr Bold
    font_visit = load_font(font_druk, 90)
    draw.text((50, 750), "ПОСЕТИТЬ", font=font_visit, fill=text_color)
    
    img.save(output_path, 'PNG')
    log(f"✅ Cover saved: {output_path}")
    return output_path

# Тест генерации обложки
test_cover = os.path.join(WORKING_DIR, "test_cover.png")
generate_cover_image("25-27", "ДЕКАБРЯ", test_cover)
log("✅ Cover generator ready")

In [ ]:
# ==========================================
# 4. BLENDER XPBD PAPER SIMULATION
# ==========================================

BLENDER_SCRIPT = '''
import math
import os
import sys
import json

import bpy

def _parse_args():
    argv = sys.argv
    if "--" not in argv:
        return {}
    args = argv[argv.index("--") + 1:]
    out = {}
    i = 0
    while i < len(args):
        k = args[i]
        if not k.startswith("--"):
            i += 1
            continue
        k = k[2:]
        v = True
        if i + 1 < len(args) and not args[i + 1].startswith("--"):
            v = args[i + 1]
            i += 1
        out[k] = v
        i += 1
    return out

def _as_int(d, k, default):
    try:
        return int(d.get(k, default))
    except:
        return default

def _as_float(d, k, default):
    try:
        return float(d.get(k, default))
    except:
        return default

def _as_str(d, k, default):
    v = d.get(k, default)
    return str(v) if v is not None else default

def setup_scene(res_x=864, res_y=1104, samples=24):
    bpy.ops.wm.read_factory_settings(use_empty=True)
    scene = bpy.context.scene
    scene.render.engine = "CYCLES"
    scene.cycles.device = "CPU"
    scene.cycles.samples = samples
    scene.render.resolution_x = res_x
    scene.render.resolution_y = res_y
    scene.render.resolution_percentage = 100
    scene.render.image_settings.file_format = "PNG"
    scene.render.image_settings.color_mode = "RGBA"
    scene.render.film_transparent = True
    scene.view_settings.view_transform = "Standard"
    scene.view_settings.look = "None"

    if not scene.world:
        scene.world = bpy.data.worlds.new("World")
    scene.world.use_nodes = True
    bg = scene.world.node_tree.nodes["Background"]
    bg.inputs[0].default_value = (0.02, 0.02, 0.02, 1)

def setup_camera_and_light(paper_width=2.0, paper_height=2.0, render_aspect=1.0):
    bpy.ops.object.camera_add(location=(0, -3.0, 0.05))
    cam = bpy.context.object
    bpy.context.scene.camera = cam
    cam.data.type = "ORTHO"
    margin = 1.05
    poster_aspect = paper_width / paper_height
    if poster_aspect > render_aspect:
        ortho_scale = (paper_width / render_aspect) * margin
    else:
        ortho_scale = paper_height * margin
    cam.data.ortho_scale = ortho_scale

    bpy.ops.object.empty_add(location=(0, 0, 0))
    target = bpy.context.object
    const = cam.constraints.new(type="DAMPED_TRACK")
    const.target = target
    const.track_axis = "TRACK_NEGATIVE_Z"

    bpy.ops.object.light_add(type="AREA", location=(-2.2, -2.4, 1.8))
    key = bpy.context.object
    key.data.energy = 450
    key.data.size = 1.6
    key.rotation_euler = (math.radians(65), math.radians(-10), math.radians(25))

    bpy.ops.object.light_add(type="AREA", location=(1.8, -2.8, 1.4))
    fill = bpy.context.object
    fill.data.energy = 150
    fill.data.size = 2.2
    fill.rotation_euler = (math.radians(70), math.radians(0), math.radians(-25))

def build_sheet(aspect, nx, ny, size=2.0):
    width = size * aspect
    height = size
    bpy.ops.mesh.primitive_grid_add(
        x_subdivisions=nx, y_subdivisions=ny, size=size,
        enter_editmode=False, location=(0, 0, 0)
    )
    obj = bpy.context.object
    obj.name = "PaperSheet"
    obj.scale[0] = width / 2.0
    obj.scale[1] = height / 2.0
    bpy.ops.object.transform_apply(location=False, rotation=False, scale=True)
    obj.rotation_euler = (math.radians(90), 0, 0)
    bpy.ops.object.transform_apply(location=False, rotation=True, scale=False)
    bpy.ops.object.mode_set(mode="EDIT")
    bpy.ops.mesh.select_all(action="SELECT")
    bpy.ops.mesh.quads_convert_to_tris(quad_method="BEAUTY", ngon_method="BEAUTY")
    bpy.ops.object.mode_set(mode="OBJECT")
    subsurf = obj.modifiers.new(name="Subsurf", type="SUBSURF")
    subsurf.levels = 1
    subsurf.render_levels = 1
    return obj

def apply_uv_map_xz(obj):
    mesh = obj.data
    if not mesh.uv_layers:
        mesh.uv_layers.new(name="UVMap")
    uv_layer = mesh.uv_layers.active.data
    xs = [v.co.x for v in mesh.vertices]
    zs = [v.co.z for v in mesh.vertices]
    min_x, max_x = min(xs), max(xs)
    min_z, max_z = min(zs), max(zs)
    dx = max(max_x - min_x, 1e-9)
    dz = max(max_z - min_z, 1e-9)
    for poly in mesh.polygons:
        for li in poly.loop_indices:
            vi = mesh.loops[li].vertex_index
            co = mesh.vertices[vi].co
            u = (co.x - min_x) / dx
            v = (co.z - min_z) / dz
            uv_layer[li].uv = (u, v)

def apply_two_sided_paper_material(obj, image_path):
    img = None
    try:
        img = bpy.data.images.load(image_path)
    except:
        img = None

    mat = bpy.data.materials.new(name="PaperMat")
    mat.use_nodes = True
    nt = mat.node_tree
    for n in list(nt.nodes):
        nt.nodes.remove(n)

    out = nt.nodes.new("ShaderNodeOutputMaterial")
    geo = nt.nodes.new("ShaderNodeNewGeometry")
    mix = nt.nodes.new("ShaderNodeMixRGB")
    mix.blend_type = "MIX"

    tex = nt.nodes.new("ShaderNodeTexImage")
    if img is not None:
        tex.image = img
        tex.interpolation = "Closest"

    nt.links.new(geo.outputs["Backfacing"], mix.inputs["Fac"])
    nt.links.new(tex.outputs["Color"], mix.inputs["Color1"])
    mix.inputs["Color2"].default_value = (1.0, 1.0, 1.0, 1.0)
    
    bsdf = nt.nodes.new("ShaderNodeBsdfPrincipled")
    bsdf.inputs["Roughness"].default_value = 0.95
    bsdf.inputs["Specular"].default_value = 0.02
    nt.links.new(mix.outputs["Color"], bsdf.inputs["Base Color"])
    nt.links.new(bsdf.outputs["BSDF"], out.inputs["Surface"])

    obj.data.materials.append(mat)

def main():
    args = _parse_args()
    poster_path = _as_str(args, "poster", None)
    out_dir = _as_str(args, "out_dir", "/tmp/blender_out")
    frame_start = _as_int(args, "frame_start", 33)
    frame_end = _as_int(args, "frame_end", 44)
    samples = _as_int(args, "samples", 24)
    nx = _as_int(args, "nx", 25)
    ny = _as_int(args, "ny", 32)
    
    os.makedirs(out_dir, exist_ok=True)
    
    res_x, res_y = 864, 1104
    setup_scene(res_x=res_x, res_y=res_y, samples=samples)
    
    poster_aspect = 1.0
    if poster_path and os.path.exists(poster_path):
        try:
            img = bpy.data.images.load(poster_path)
            poster_aspect = float(img.size[0]) / max(1.0, float(img.size[1]))
        except:
            poster_aspect = 1.0
    
    sheet = build_sheet(aspect=poster_aspect, nx=nx, ny=ny, size=2.0)
    apply_uv_map_xz(sheet)
    
    paper_width = 2.0 * poster_aspect
    paper_height = 2.0
    render_aspect = float(res_x) / float(res_y)
    
    setup_camera_and_light(paper_width=paper_width, paper_height=paper_height, render_aspect=render_aspect)
    apply_two_sided_paper_material(sheet, image_path=poster_path if poster_path else "")
    bpy.ops.object.shade_smooth()
    
    # Simplified physics - render frames from folded to unfolded
    from mathutils import Vector
    import random
    
    mesh = sheet.data
    rest = [v.co.copy() for v in mesh.vertices]
    
    rnd = random.Random(7)
    x = [p.copy() for p in rest]
    
    # Apply crumple deformation
    for i in range(len(x)):
        x[i].y += rnd.uniform(-0.3, 0.3)
        x[i].z += rnd.uniform(-0.2, 0.2)
        x[i].x += rnd.uniform(-0.1, 0.1)
    
    scene = bpy.context.scene
    scene.frame_start = frame_start
    scene.frame_end = frame_end
    
    for f in range(frame_start, frame_end + 1):
        # Interpolate from crumpled to flat
        t = (f - frame_start) / max(1, frame_end - frame_start)
        t = t ** 0.5  # ease out
        
        for i, vert in enumerate(mesh.vertices):
            vert.co = x[i] * (1 - t) + rest[i] * t
        mesh.update()
        bpy.context.view_layer.update()
        
        scene.frame_current = f
        scene.render.filepath = os.path.join(out_dir, f"frame_{f:04d}.png")
        bpy.ops.render.render(write_still=True)
        print(f"Rendered frame {f}")

if __name__ == "__main__":
    main()
'''

# Сохраняем скрипт
BLENDER_SCRIPT_PATH = os.path.join(WORKING_DIR, "blender_xpbd.py")
with open(BLENDER_SCRIPT_PATH, 'w') as f:
    f.write(BLENDER_SCRIPT)

log("✅ Blender script saved")

In [ ]:
# ==========================================
# 5. SHOWREEL ASSEMBLY
# ==========================================

def ensure_dir(p: Path):
    Path(p).mkdir(parents=True, exist_ok=True)

def read_bgr(path):
    img = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(path)
    return img

def write_bgr(path, bgr):
    ensure_dir(Path(path).parent)
    cv2.imwrite(str(path), bgr)

def make_background(h: int, w: int):
    """Dark gradient background."""
    yy, xx = np.mgrid[0:h, 0:w].astype(np.float32)
    cx, cy = w * 0.5, h * 0.52
    r = np.sqrt((xx - cx) ** 2 + (yy - cy) ** 2)
    r = r / (max(h, w) * 0.9)
    r = np.clip(r, 0.0, 1.0)
    base = 14.0 + (1.0 - r) * 18.0
    bg = np.clip(base, 0, 255).astype(np.uint8)
    return np.dstack([bg, bg, bg])

def composite_on_bg(rgba_path):
    """Composite RGBA frame onto dark background."""
    rgba = cv2.imread(str(rgba_path), cv2.IMREAD_UNCHANGED)
    if rgba is None or rgba.ndim != 3 or rgba.shape[2] < 4:
        return
    bgr = rgba[:, :, :3].astype(np.float32)
    a = (rgba[:, :, 3].astype(np.float32) / 255.0)[:, :, None]
    h, w = bgr.shape[:2]
    bg = make_background(h, w).astype(np.float32)
    out = bgr * a + bg * (1.0 - a)
    cv2.imwrite(str(rgba_path), np.clip(out, 0, 255).astype(np.uint8))

def ease_in_out_back(x):
    """Dramatic easing with overshoot."""
    c1 = 1.70158
    c2 = c1 * 1.525
    if x < 0.5:
        return (pow(2 * x, 2) * ((c2 + 1) * 2 * x - c2)) / 2
    return (pow(2 * x - 2, 2) * ((c2 + 1) * (x * 2 - 2) + c2) + 2) / 2

def render_motion(base_frames, length, reverse=False):
    """Sample frames with easing."""
    if length <= 0:
        return []
    if len(base_frames) == 1:
        return [base_frames[0]] * int(length)
    
    out = []
    n = int(length)
    for i in range(n):
        t = 0.0 if n == 1 else (i / (n - 1))
        s = float(max(0.0, min(1.0, ease_in_out_back(t)))) * (len(base_frames) - 1)
        if reverse:
            s = (len(base_frames) - 1) - s
        idx = int(round(s))
        idx = max(0, min(len(base_frames) - 1, idx))
        out.append(base_frames[idx])
    return out

log("✅ Showreel functions ready")

In [ ]:
# ==========================================
# 6. AUDIO INTEGRATION
# ==========================================

def add_audio(video_path, audio_path, output_path, start_sec=70):
    """Mix audio starting at specified timestamp with video."""
    if not os.path.exists(audio_path):
        log(f"⚠️ Audio not found: {audio_path}")
        shutil.copy(video_path, output_path)
        return
    
    cmd = [
        "ffmpeg", "-y",
        "-ss", str(float(start_sec)),
        "-i", str(audio_path),
        "-i", str(video_path),
        "-shortest",
        "-c:v", "copy",
        "-c:a", "aac",
        "-b:a", "192k",
        str(output_path)
    ]
    subprocess.check_call(cmd)
    log(f"✅ Audio added: {output_path}")

def render_video_from_frames(frames_dir, output_path, fps=24):
    """Use FFmpeg to assemble frames into video."""
    cmd = [
        "ffmpeg", "-y",
        "-framerate", str(fps),
        "-i", os.path.join(frames_dir, "frame_%06d.png"),
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        "-crf", "18",
        str(output_path)
    ]
    subprocess.check_call(cmd)
    log(f"✅ Video rendered: {output_path}")

log("✅ Audio functions ready")

In [ ]:
# ==========================================
# 7. MAIN PIPELINE
# ==========================================

def run_blender_render(poster_path, out_dir):
    """Run Blender in background to render frames."""
    cmd = [
        "blender", "-b", "-noaudio",
        "-P", BLENDER_SCRIPT_PATH,
        "--",
        "--poster", str(poster_path),
        "--out_dir", str(out_dir),
        "--frame_start", str(FRAME_START),
        "--frame_end", str(FRAME_END),
        "--samples", str(BLENDER_SAMPLES)
    ]
    log(f"🎬 Rendering: {os.path.basename(poster_path)}")
    subprocess.check_call(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    log(f"✅ Done: {os.path.basename(poster_path)}")

async def main_pipeline():
    # 1. Load payload
    payload_path = os.path.join(SOURCE_FOLDER, "payload.json")
    if not os.path.exists(payload_path):
        candidates = glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "**/payload.json"), recursive=True)
        if candidates:
            payload_path = candidates[0]
    
    if os.path.exists(payload_path):
        log(f"📄 Loading: {payload_path}")
        with open(payload_path, 'r', encoding='utf-8') as f:
            raw = json.load(f)
    else:
        log("⚠️ Using default data")
        raw = DEFAULT_RAW_DATA
    
    intro = raw.get("intro", {})
    scenes = raw.get("scenes", [])
    
    # 2. Parse dates for cover
    date_str = intro.get("date", "25-27 декабря")
    parts = date_str.split()
    date_range = parts[0] if parts else "1-3"
    month = parts[1].upper() if len(parts) > 1 else "ЯНВАРЯ"
    
    # 3. Generate cover
    cover_path = os.path.join(WORKING_DIR, "cover.png")
    generate_cover_image(date_range, month, cover_path)
    
    # 4. Collect poster images
    poster_paths = [cover_path]  # Cover first
    for idx, scene in enumerate(scenes[:4]):  # Max 4 posters
        images = scene.get("images", [])
        if images:
            url = images[0]
            local_path = os.path.join(WORKING_DIR, f"poster_{idx}.jpg")
            if url.startswith("http"):
                # Download
                import requests
                try:
                    r = requests.get(url, timeout=30)
                    if r.status_code == 200:
                        with open(local_path, 'wb') as f:
                            f.write(r.content)
                        poster_paths.append(local_path)
                except:
                    pass
            elif os.path.exists(url):
                poster_paths.append(url)
    
    log(f"📷 Posters to render: {len(poster_paths)}")
    
    # 5. Render each poster with Blender
    all_frames = []
    for i, poster in enumerate(poster_paths):
        frames_dir = os.path.join(WORKING_DIR, f"frames_{i}")
        os.makedirs(frames_dir, exist_ok=True)
        
        run_blender_render(poster, frames_dir)
        
        # Load frames
        frames = []
        for f in range(FRAME_START, FRAME_END + 1):
            frame_path = os.path.join(frames_dir, f"frame_{f:04d}.png")
            if os.path.exists(frame_path):
                composite_on_bg(frame_path)
                frames.append(read_bgr(frame_path))
        
        if frames:
            # Unfold animation (crumpled -> flat)
            unfold = render_motion(frames, 24, reverse=False)
            # Hold flat
            hold = [frames[-1]] * 30
            # Fold animation (flat -> crumpled)
            fold = render_motion(frames, 24, reverse=True)
            # Hold ball
            ball_hold = [frames[0]] * 6
            
            if i == 0:
                # Cover: start flat, then fold
                all_frames.extend(hold)
                all_frames.extend(fold)
                all_frames.extend(ball_hold)
            else:
                # Regular poster: unfold, hold, fold
                all_frames.extend(unfold)
                all_frames.extend(hold)
                all_frames.extend(fold)
                all_frames.extend(ball_hold)
    
    # 6. Save sequence frames
    seq_dir = os.path.join(WORKING_DIR, "sequence")
    os.makedirs(seq_dir, exist_ok=True)
    
    for i, frame in enumerate(all_frames, start=1):
        write_bgr(os.path.join(seq_dir, f"frame_{i:06d}.png"), frame)
    
    log(f"🎞️ Total frames: {len(all_frames)}")
    
    # 7. Render video
    video_no_audio = os.path.join(WORKING_DIR, "crumple_video.mp4")
    render_video_from_frames(seq_dir, video_no_audio, fps=FPS)
    
    # 8. Add audio
    audio_path = None
    # Search for audio file
    audio_candidates = glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "**", AUDIO_FILENAME), recursive=True)
    audio_candidates.extend(glob.glob(os.path.join(SOURCE_FOLDER, "**", AUDIO_FILENAME), recursive=True))
    if audio_candidates:
        audio_path = audio_candidates[0]
    
    final_video = os.path.join(WORKING_DIR, "crumple_video_final.mp4")
    if audio_path:
        add_audio(video_no_audio, audio_path, final_video, start_sec=AUDIO_START_SEC)
    else:
        log("⚠️ Audio file not found, video without sound")
        shutil.copy(video_no_audio, final_video)
    
    log(f"\n🎉 DONE! Output: {final_video}")
    return final_video

# RUN
import asyncio
final_path = await main_pipeline()
log(f"\n✅ Video ready: {final_path}")